In [1]:
import pandas as pd
import numpy as np
import requests
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install lxml

In [2]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

In [3]:
df= df[df['Borough']!='Not assigned']

In [4]:
df.shape
df['Postal Code'].nunique()

103

In [5]:
geo_data=pd.read_csv("https://cocl.us/Geospatial_data")
geo_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [6]:
geo_data.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df.rename(columns={'Postal Code':'PostalCode'},inplace=True)


In [161]:
df1=pd.merge(df,geo_data,on='PostalCode',how='left')

In [162]:
df1

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [9]:
df1.loc[df1['PostalCode']=='M3A']


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656


In [10]:
address = 'Toronto'

geolocator = Nominatim(user_agent="coursera")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [11]:
my_map = folium.Map(location=[latitude, longitude], zoom_start=11)


for lat, lng, label in zip(df1['Latitude'], df1['Longitude'], df1['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

In [12]:
CLIENT_ID = 'EF4E1ZIUELR5TD14J0GK1WNC5Y3BJ3WABAZVYKUH0Y3V5ZWK' # Foursquare ID
CLIENT_SECRET = '3ZHBUQSJIHQVNWV52VSJT2DIOILHXGZG1WQLQYCOSAVAQXAI' # Foursquare Secret
VERSION = '20180605' # API version

In [13]:
df1.set_index('PostalCode', inplace = True) 

In [14]:
etobicoke= df1[df1['Borough']=='Etobicoke'].reset_index(drop=True)

In [15]:
address = 'Etobicoke'

geolocator = Nominatim(user_agent="coursera")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)


for lat, lng, label in zip(etobicoke['Latitude'], etobicoke['Longitude'], etobicoke['Neighbourhood']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinate of Etobicoke are 43.6435559, -79.5656326.


In [25]:

neighborhood_latitude = etobicoke.loc[6, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = etobicoke.loc[6, 'Longitude'] # neighborhood longitude value

neighborhood_name = etobicoke.loc[6, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens are 43.739416399999996, -79.5884369.


In [26]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
print(url) # display URL


https://api.foursquare.com/v2/venues/explore?&client_id=EF4E1ZIUELR5TD14J0GK1WNC5Y3BJ3WABAZVYKUH0Y3V5ZWK&client_secret=3ZHBUQSJIHQVNWV52VSJT2DIOILHXGZG1WQLQYCOSAVAQXAI&v=20180605&ll=43.739416399999996,-79.5884369&radius=500&limit=100


In [27]:
results = requests.get(url).json()

In [29]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
venues = results['response']['groups'][0]['items']
nearby_venues= pd.json_normalize(venues)

In [45]:
nearby_venues
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues[filtered_columns]

In [48]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type,axis=1)

In [60]:
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]
 

In [62]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [63]:
venues = getNearbyVenues(names=df1['Borough'],latitudes=df1['Latitude'],longitudes=df1['Longitude'])

North York
North York
Downtown Toronto
North York
Downtown Toronto
Etobicoke
Scarborough
North York
East York
Downtown Toronto
North York
Etobicoke
Scarborough
North York
East York
Downtown Toronto
York
Etobicoke
Scarborough
East Toronto
Downtown Toronto
York
Scarborough
East York
Downtown Toronto
Downtown Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East York
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
West Toronto
Scarborough
North York
North York
East Toronto
Downtown Toronto
North York
North York
Scarborough
North York
North York
East Toronto
North York
York
North York
Scarborough
North York
North York
Central Toronto
Central Toronto
York
York
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Etobicoke
Scarborough
North York
Central Toronto
Central Toronto
West Toronto
Mississauga
Etobicoke
Scarborough
Central Toronto
Downtown Toronto
West Toron

In [64]:
venues.shape

(2146, 7)

In [66]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,111,111,111,111,111,111
Downtown Toronto,1245,1245,1245,1245,1245,1245
East Toronto,122,122,122,122,122,122
East York,77,77,77,77,77,77
Etobicoke,69,69,69,69,69,69
Mississauga,13,13,13,13,13,13
North York,237,237,237,237,237,237
Scarborough,94,94,94,94,94,94
West Toronto,160,160,160,160,160,160


In [67]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 264 unique categories.


In [68]:
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()

In [72]:
grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.009009,0.000000,...,0.00000,0.009009,0.000000,0.009009,0.000000,0.000000,0.000000,0.000000,0.000000,0.009009
1,Downtown Toronto,0.000000,0.000803,0.000803,0.000803,0.001606,0.00241,0.001606,0.014458,0.001606,...,0.00241,0.011245,0.000803,0.003213,0.000000,0.006426,0.000803,0.000000,0.000803,0.006426
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.024590,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.008197,0.000000,0.000000,0.000000,0.024590
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.012987,0.000000,0.000000,0.000000,0.000000,0.012987
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.014493,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014493,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.076923,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004219,0.000000,0.004219,0.000000,0.000000,0.00000,0.000000,0.008439,0.000000,...,0.00000,0.000000,0.004219,0.012658,0.000000,0.000000,0.000000,0.000000,0.004219,0.000000
7,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.010638,0.000000,...,0.00000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.006250,...,0.00000,0.012500,0.000000,0.012500,0.000000,0.006250,0.000000,0.000000,0.000000,0.012500
9,York,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000


In [73]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Park  0.05
3            Café  0.05
4     Pizza Place  0.05


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.11
1                 Café  0.05
2           Restaurant  0.03
3  Japanese Restaurant  0.03
4                Hotel  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.06
2                Café  0.04
3             Brewery  0.04
4  Italian Restaurant  0.04


----East York----
                    venue  freq
0             Coffee Shop  0.06
1                    Bank  0.05
2  Furniture / Home Store  0.04
3     Sporting Goods Shop  0.04
4            Burger Joint  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.12
1  Sandwich Place  0.07
2     Coffee Shop  0.06
3   Grocery Store  0.04
4             Gym  0.04


----Mississauga----
                       venue  freq
0      

In [84]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [88]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Clothing Store,Sushi Restaurant,Pub,Dessert Shop,Restaurant
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Clothing Store
2,East Toronto,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Park,Restaurant,Ice Cream Shop,Yoga Studio,Pub
3,East York,Coffee Shop,Bank,Sporting Goods Shop,Sandwich Place,Furniture / Home Store,Pizza Place,Burger Joint,Park,Shopping Mall,Supermarket
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Fast Food Restaurant,Pharmacy,Grocery Store,Gym,Discount Store,Fried Chicken Joint,Bakery
5,Mississauga,Coffee Shop,Hotel,Intersection,Gym,Fried Chicken Joint,Mediterranean Restaurant,American Restaurant,Burrito Place,Gas Station,Sandwich Place
6,North York,Coffee Shop,Clothing Store,Restaurant,Bank,Grocery Store,Pizza Place,Japanese Restaurant,Park,Fast Food Restaurant,Sandwich Place
7,Scarborough,Chinese Restaurant,Coffee Shop,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Pizza Place,Intersection,Fried Chicken Joint,Soccer Field
8,West Toronto,Café,Bar,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Pizza Place,Park,Breakfast Spot,Bookstore
9,York,Park,Convenience Store,Playground,Trail,Caribbean Restaurant,Field,Sandwich Place,Pool,Hockey Arena,Discount Store


In [93]:
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 3, 4, 0, 1, 3, 1, 2], dtype=int32)

In [94]:
kmeans.labels_

array([1, 1, 1, 3, 4, 0, 1, 3, 1, 2], dtype=int32)

In [142]:
grouped=grouped.drop('Cluster Labels',1)
merged= grouped

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.009009,0.000000,...,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Clothing Store,Sushi Restaurant,Pub,Dessert Shop,Restaurant
1,Downtown Toronto,0.0,0.000803,0.000803,0.000803,0.001606,0.00241,0.001606,0.014458,0.001606,...,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Clothing Store
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.024590,0.000000,...,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Park,Restaurant,Ice Cream Shop,Yoga Studio,Pub
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,Coffee Shop,Bank,Sporting Goods Shop,Sandwich Place,Furniture / Home Store,Pizza Place,Burger Joint,Park,Shopping Mall,Supermarket
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.014493,0.000000,...,Pizza Place,Sandwich Place,Coffee Shop,Fast Food Restaurant,Pharmacy,Grocery Store,Gym,Discount Store,Fried Chicken Joint,Bakery


In [146]:
merged=merged.drop(['Cluster Labels'],1)

In [149]:
merged.insert(0, 'Cluster Labels', kmeans.labels_)

In [150]:
merged

,Cluster Labels,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.009009,...,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Clothing Store,Sushi Restaurant,Pub,Dessert Shop,Restaurant
1,1,Downtown Toronto,0.000000,0.000803,0.000803,0.000803,0.001606,0.00241,0.001606,0.014458,...,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Clothing Store
2,1,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.024590,...,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Park,Restaurant,Ice Cream Shop,Yoga Studio,Pub
3,3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,Coffee Shop,Bank,Sporting Goods Shop,Sandwich Place,Furniture / Home Store,Pizza Place,Burger Joint,Park,Shopping Mall,Supermarket
4,4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.014493,...,Pizza Place,Sandwich Place,Coffee Shop,Fast Food Restaurant,Pharmacy,Grocery Store,Gym,Discount Store,Fried Chicken Joint,Bakery
5,0,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.076923,...,Coffee Shop,Hotel,Intersection,Gym,Fried Chicken Joint,Mediterranean Restaurant,American Restaurant,Burrito Place,Gas Station,Sandwich Place
6,1,North York,0.004219,0.000000,0.004219,0.000000,0.000000,0.00000,0.000000,0.008439,...,Coffee Shop,Clothing Store,Restaurant,Bank,Grocery Store,Pizza Place,Japanese Restaurant,Park,Fast Food Restaurant,Sandwich Place
7,3,Scarborough,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.010638,...,Chinese Restaurant,Coffee Shop,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Pizza Place,Intersection,Fried Chicken Joint,Soccer Field
8,1,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,Café,Bar,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Pizza Place,Park,Breakfast Spot,Bookstore
9,2,York,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,Park,Convenience Store,Playground,Trail,Caribbean Restaurant,Field,Sandwich Place,Pool,Hockey Arena,Discount Store


In [189]:
latitude

43.6435559

In [181]:
df2=df1.groupby('Borough').mean().reset_index()
df2=df2.rename(columns={'Borough':'Neighborhood'})

In [187]:
df2=df2.join(merged.set_index('Neighborhood'), on='Neighborhood')

In [188]:
df2

,Neighborhood,Latitude,Longitude,Cluster Labels,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,43.701980,-79.398954,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Clothing Store,Sushi Restaurant,Pub,Dessert Shop,Restaurant
1,Downtown Toronto,43.654597,-79.383972,1,0.000000,0.000803,0.000803,0.000803,0.001606,0.00241,...,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Clothing Store
2,East Toronto,43.669436,-79.324654,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Park,Restaurant,Ice Cream Shop,Yoga Studio,Pub
3,East York,43.700303,-79.335851,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,Coffee Shop,Bank,Sporting Goods Shop,Sandwich Place,Furniture / Home Store,Pizza Place,Burger Joint,Park,Shopping Mall,Supermarket
4,Etobicoke,43.660043,-79.542074,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,Pizza Place,Sandwich Place,Coffee Shop,Fast Food Restaurant,Pharmacy,Grocery Store,Gym,Discount Store,Fried Chicken Joint,Bakery
5,Mississauga,43.636966,-79.615819,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,Coffee Shop,Hotel,Intersection,Gym,Fried Chicken Joint,Mediterranean Restaurant,American Restaurant,Burrito Place,Gas Station,Sandwich Place
6,North York,43.750727,-79.429338,1,0.004219,0.000000,0.004219,0.000000,0.000000,0.00000,...,Coffee Shop,Clothing Store,Restaurant,Bank,Grocery Store,Pizza Place,Japanese Restaurant,Park,Fast Food Restaurant,Sandwich Place
7,Scarborough,43.766229,-79.249085,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,Chinese Restaurant,Coffee Shop,Breakfast Spot,Bakery,Bank,Fast Food Restaurant,Pizza Place,Intersection,Fried Chicken Joint,Soccer Field
8,West Toronto,43.652653,-79.449290,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,Café,Bar,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Pizza Place,Park,Breakfast Spot,Bookstore
9,York,43.690797,-79.472633,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,Park,Convenience Store,Playground,Trail,Caribbean Restaurant,Field,Sandwich Place,Pool,Hockey Arena,Discount Store


In [190]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df2['Latitude'], df2['Longitude'], df2['Neighborhood'], df2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters